In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings


def make_chroma_db(documents):
    # Chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    docs = splitter.split_documents(documents)

    # 벡터 저장소 만들기
    db = Chroma.from_documents(docs, OpenAIEmbeddings(), persist_directory="chroma_db")
    return db

def get_top5_docs_from_db(query):
    db = Chroma(persist_directory="chroma_db", embedding_function=OpenAIEmbeddings())
    retriever = db.as_retriever(search_kwargs={"k": 5}) # 상위 5개만 추출하도록 설정

    return retriever.get_relevant_documents(query)


In [2]:
from openai import OpenAI

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
client = OpenAI()

# 🔍 GPT를 사용해 요약 생성
def summarize_with_gpt(content: str, file_path: str, max_chars: int = 1500) -> str:
    prompt = f"""
    다음은 '{file_path}'라는 파일의 코드입니다. 
    이 파일의 목적이 무엇인지, 어떤 기능이 있고 어떤 문제를 해결하는지 간단히 요약해 주세요. 
    \n\n```python\n{content[:max_chars]}\n```\n\n요약:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",  # 또는 gpt-3.5-turbo
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ GPT 요약 실패 ({file_path}): {e}")
        return "요약 실패"

In [3]:
import sys
sys.path.append(r"C:\Users\USER\Desktop\GitHub\3rd_project")  # chahae 폴더의 상위 폴더

from chahae.github_repo_viewer import main
from dotenv import load_dotenv
import os

load_dotenv()
documents = main(os.environ.get("GITHUB_TOKEN"))


[설치] python-dotenv 패키지를 설치합니다...
[설치] python-dotenv 패키지 설치 완료
[Git] Git이 설치되어 있습니다: C:\Program Files\Git\cmd\git.EXE

[정보] 저장소 소유자: hwangchahae
[정보] 저장소 이름: coding_test_study

[정보] 전체 저장소 내용을 가져오는 중...


In [4]:
chroma_db = make_chroma_db(documents)

C:\Users\USER\AppData\Local\Temp\ipykernel_10732\2972093822.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = Chroma.from_documents(docs, OpenAIEmbeddings(), persist_directory="chroma_db")


In [5]:
query = "큐 관련 문제 코드 찾아줘"
results = get_top5_docs_from_db(query)
results

C:\Users\USER\AppData\Local\Temp\ipykernel_10732\2972093822.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="chroma_db", embedding_function=OpenAIEmbeddings())
C:\Users\USER\AppData\Local\Temp\ipykernel_10732\2972093822.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)


[Document(metadata={'file_name': 'README.md', 'source': 'week_10/1697_숨바꼭질/README.md'}, page_content='# 숨바꼭질 문제 풀이\n\n백준 문제 번호: 1697'),
 Document(metadata={'source': 'week_8/15683_감시/README.md', 'file_name': 'README.md'}, page_content='# 감시 문제 풀이\n\n백준 문제 번호: 15683'),
 Document(metadata={'source': 'README.md', 'file_name': 'README.md'}, page_content='# 알고리즘 풀이 (2025.03.27 ~)\n\n## 📌 이번 주 문제'),
 Document(metadata={'source': 'README.md', 'file_name': 'README.md'}, page_content='## 📌 스터디 방식\n1. 풀 플랫폼은 [백준](https://www.acmicpc.net/)을 사용한다.\n2. 알고리즘 스터디는 정해진 문제를 푼다.\n3. 각자 문제 풀이 후 스터디 모임 때 코드 리뷰를 진행한다.\n4. 스터디 진행 방식 참고: [코딩테스트 진행 방식](https://dev-dain.tistory.com/155)\n\n## 🏆 문제 풀이 기록'),
 Document(metadata={'file_name': 'README.md', 'source': 'week_10/1012_유기농_배추/README.md'}, page_content='# 유기농 배추 문제 풀이\n\n백준 문제 번호: 1012')]

In [6]:
print(results[0].metadata)

{'file_name': 'README.md', 'source': 'week_10/1697_숨바꼭질/README.md'}


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain


# 4. LLM 준비
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# 5. 문서를 기반으로 설명 생성 (Chain 사용)
qa_chain = load_qa_with_sources_chain(llm, chain_type="stuff")
result = qa_chain({"input_documents": results, "question": query}, return_only_outputs=True)

# 6. 출력
print("📝 설명 결과:\n", result["output_text"])
# print("\n📚 참고된 문서 정보:\n", result["sources"])

C:\Users\USER\AppData\Local\Temp\ipykernel_10732\1783291465.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0)
C:\Users\USER\AppData\Local\Temp\ipykernel_10732\1783291465.py:9: LangChainDeprecationWarning: This function is deprecated. Refer to this guide on retrieval and question answering with sources: https://python.langchain.com/docs/how_to/qa_sources/
See also the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.

📝 설명 결과:
 I'm sorry, but the provided contents do not contain any information about the code for a problem related to "큐".
SOURCES:


In [8]:
print(result["output_text"])

I'm sorry, but the provided contents do not contain any information about the code for a problem related to "큐".
SOURCES:


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import Runnable
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain_core.documents import Document

class MemoryRAGRetriever:
    def __init__(self, collection: Chroma):
        self.vectorstore = collection
        self.retriever = self.vectorstore.as_retriever()
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

        self.retriever_with_memory = create_history_aware_retriever(
            llm=self.llm,
            retriever=self.retriever,
            prompt=PromptTemplate.from_template("""
다음은 사용자와의 최근 대화 일부입니다:
{chat_history}

사용자의 질문: {input}

위 질문에 가장 관련된 내용을 찾기 위한 쿼리를 만들어주세요.
""")
        )

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=self.retriever_with_memory,
            return_source_documents=True
        )
        self.chat_history = ChatMessageHistory()

    def query(self, user_query: str) -> str:
        # 최근 3쌍만 유지 (user + ai = 6개)
        recent_history = self.chat_history.messages[-6:]
        
        result = self.qa_chain.invoke({
            "input": user_query,
            "chat_history": recent_history
        })

        self.chat_history.add_user_message(user_query)
        self.chat_history.add_ai_message(result["result"])
        return result["result"]
